# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import and download nltk package
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Import libraries
import pandas as pd
import pprint

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertTableName', con=engine)
df.head()
X = df['message']
Y = df[df.columns[4:]]


### 2. Write a tokenization function to process your text data

In [4]:
# Create function that tokenizes text input
def tokenize(text):
    '''
    Function splitting messages into words, converting to lower case and removing punctuation
    Args: text = message in form of string
    Return: clean_tokens = list of cleaned tokens
    '''
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens   
       

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Create pipeline that uses CountVectorizer, a TfidfTransformer and then classifies the message via RandomForstClassier
# The predictor is supplemented by the MultiOutputClassifier to ensure that multiple target variables are predicted
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('randomf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [7]:
# Training the machine learning pipeline
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Convert y_pred from array to dataframe
y_pred = pd.DataFrame(y_pred, columns=df.columns[4:])


In [9]:
# Iterate through columns of y_pred and y_test and calculate precision, recall and f1_score
pp = pprint.PrettyPrinter(indent=4)

for col in y_pred.columns:
    print(col)
    pp.pprint(classification_report(y_test[col], y_pred[col], output_dict=True))
    

related
{   '0.0': {   'f1-score': 0.6178073894609328,
               'precision': 0.6343283582089553,
               'recall': 0.602125147579693,
               'support': 847},
    '1.0': {   'f1-score': 0.814357163871727,
               'precision': 0.8041836141778036,
               'recall': 0.8247914183551848,
               'support': 1678},
    'accuracy': 0.7500990099009901,
    'macro avg': {   'f1-score': 0.71608227666633,
                     'precision': 0.7192559861933794,
                     'recall': 0.713458282967439,
                     'support': 2525},
    'weighted avg': {   'f1-score': 0.7484254177624428,
                        'precision': 0.7472064253438968,
                        'recall': 0.7500990099009901,
                        'support': 2525}}
request
{   '0.0': {   'f1-score': 0.8604651162790697,
               'precision': 0.8105147864184009,
               'recall': 0.9169764560099133,
               'support': 1614},
    '1.0': {   'f1-score': 0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'recall': 1.0,
               'support': 2486},
    '1.0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 39},
    'accuracy': 0.9845544554455445,
    'macro avg': {   'f1-score': 0.49610856116543606,
                     'precision': 0.49227722772277227,
                     'recall': 0.5,
                     'support': 2525},
    'weighted avg': {   'f1-score': 0.9768917885602171,
                        'precision': 0.9693474757376727,
                        'recall': 0.9845544554455445,
                        'support': 2525}}
death
{   '0.0': {   'f1-score': 0.9889845784097736,
               'precision': 0.9782091917591125,
               'recall': 1.0,
               'support': 2469},
    '1.0': {   'f1-score': 0.03508771929824561,
               'precision': 1.0,
               'recall': 0.017857142857142856,
               'support': 56},
    'accuracy': 0.9782178217821782,
    'macro avg': {   'f1-score': 0.5120361488540096,
                     'precision':

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Get list of all parameters that can be used for GridSearchCV
pipeline.get_params().keys()


dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'randomf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'randomf__estimator__bootstrap', 'randomf__estimator__class_weight', 'randomf__estimator__criterion', 'randomf__estimator__max_depth', 'randomf__estimator__max_features', 'randomf__estimator__max_leaf_nodes', 'randomf__estimator__min_impurity_decrease', 'randomf__estimator__min_impurity_split', 'randomf__estimator__min_samples_leaf', 'randomf__estimator__min_samples_split', 'randomf__estimator__min_weight_fraction_leaf', 'randomf__estimator__n_estimators', 'randomf__estimator__n_jobs', 'randomf__estimator__oob_score', 'rand

In [11]:
# Define parameters to be used for grid search
parameters = {
    'vect__ngram_range': ((1,1), (1,2)),
    #'vect__max_df': [0.5, 0.7, 0.9, 1],
    'tfidf__use_idf': [True, False],
    #'randomf__estimator__n_estimators': [10, 30, 60],
    'randomf__estimator__max_depth': [5, 7, 9, 11],
    'randomf__estimator__min_samples_split': [4, 6, 8, 10]
}

# Instantiate GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)

# Fit the grid search model and return the predictions for the optimal parameter combination
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Iterate through columns of y_pred and y_test and calculate precision, recall and f1_score
pp = pprint.PrettyPrinter(indent=4)

for col in y_pred.columns:
    print(col)
    pp.pprint(classification_report(y_test[col], y_pred[col], output_dict=True))
    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Create alternative pipeline with different machine learning algorithm
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('randomf', MultiOutputClassifier(AdaBoostClassifier()))
])

# Training the machine learning pipeline
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# Convert y_pred from array to dataframe
y_pred = pd.DataFrame(y_pred, columns=df.columns[4:])


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.